# PNREC Presentation Prep



In [80]:
#prelims
import numpy as np
import polars as pl
import pandas as pd
import geopandas as gpd

import plotly.express as px
import datetime as dt
from dateutil.relativedelta import relativedelta
import geopy
from geopy.geocoders import Bing
from geopy.extra.rate_limiter import RateLimiter

#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data and initial prep

Data used here is processed in the geodata_prep and port_stats notebooks in the main directory. 

The data used in the app is a dataframe with reach row corresponding to a port call, including data such as port, dock, and vessel info, time of arrival, hours at berth and at anchor, time in port waters, etc. 

In [81]:
#create main dataframe
calls_df = (
    #read in data
    pl.read_parquet('../dashboard/calls.parquet')
    #get year month and date from arrival time
    .with_columns(
        pl.col('time_arrival').dt.year().alias('year'),
        pl.col('time_arrival').dt.date().dt.month_start().alias('month'),
        pl.col('time_arrival').dt.date().alias('date')
    )
    #add port group for NWSPA and San Pedro Ports
    .with_columns(
        pl.when(pl.col('port_name').is_in(['Seattle, WA', 'Tacoma, WA']))
        .then(pl.lit('NW Seaport Alliance'))
        .when(pl.col('port_name').is_in(['Port of Los Angeles, CA',
                                         'Port of Long Beach, CA']))
        .then(pl.lit('San Pedro Ports'))
        .otherwise(pl.col('port_name'))
        .alias('port_group')
    )
)
#get date bounds
earliest_date = calls_df['time_arrival'].min().date()
latest_date = calls_df['time_arrival'].max().date()

#inspect data
calls_df.describe()

statistic,call_id,port_name,port_lat,port_lon,dock_name,dock_id,facility_type,dock_lat,dock_lon,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters,year,month,date,port_group
str,str,str,f64,f64,str,str,str,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,f64,f64,f64,str,str,str
"""count""","""156205""","""156205""",156205.0,156205.0,"""156205""","""156205""","""155832""",156205.0,156205.0,156205.0,156205.0,"""156205""","""156205""","""156205""","""156205""",156205.0,156205.0,156205.0,156205.0,156205.0,156205.0,"""156205""","""156205""","""156205"""
"""null_count""","""0""","""0""",0.0,0.0,"""0""","""0""","""373""",0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0"""
"""mean""",null,null,32.686926,-95.288618,null,null,null,32.686999,-95.287408,1.0116e7,208.089351,"""2021-07-26 08:52:21.130284""","""2021-07-26 20:08:02.973278""","""2021-07-29 00:46:41.506270""","""2021-07-29 05:56:08.875797""",47.656672,8.141756,11.253709,5.149874,69.055084,2021.070702,"""2021-07-11 13:41:40.076000""","""2021-07-26 07:35:18.562000""",null
"""std""",null,null,7.207185,20.905249,null,null,null,7.207342,20.905297,2.6094e7,58.853774,null,null,null,null,62.115883,28.449073,34.912037,40.933445,95.135948,2.030186,null,null,null
"""min""","""0_Corpus Christi, TX_2020-04-0…","""Albany Port District, NY""",17.938939,-166.549916,"""ADM Corpus Christi Grain Eleva…","""00XE""","""Anchorage""",17.936081,-166.53444,0.0,101.0,"""2018-01-01 00:35:19""","""2018-01-01 00:35:19""","""2018-01-01 04:23:54""","""2018-01-01 09:15:57""",0.083333,0.0,0.0,0.0,0.083333,2018.0,"""2018-01-01""","""2018-01-01""","""Albany Port District, NY"""
"""25%""",null,null,28.96133,-118.2095,null,null,null,28.936819,-118.217936,9.294977e6,177.0,"""2019-09-24 13:52:02""","""2019-09-24 21:12:08""","""2019-09-26 17:14:26""","""2019-09-26 20:53:34""",16.45,0.0,2.583333,2.083333,25.516667,2019.0,"""2019-09-01""","""2019-09-24""",null
"""50%""",null,null,30.69123,-90.085256,null,null,null,30.723889,-90.12417,9.401491e6,190.0,"""2021-09-06 09:40:43""","""2021-09-07 02:34:37""","""2021-09-09 04:35:42""","""2021-09-09 11:20:56""",30.85,0.0,3.483333,2.75,43.466667,2021.0,"""2021-09-01""","""2021-09-07""",null
"""75%""",null,null,37.82152,-80.117801,null,null,null,37.797222,-80.115556,9.615042e6,231.0,"""2023-05-08 10:41:45""","""2023-05-08 22:48:08""","""2023-05-11 09:53:50""","""2023-05-11 13:02:31""",56.983333,0.0,5.3,3.5,80.233333,2023.0,"""2023-05-01""","""2023-05-08""",null
"""max""","""9993808_Honolulu, O'ahu, HI_20…","""Wilmington, NC""",61.23778,-66.096678,"""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""Tie Off""",61.24306,-66.086926,9.800025e8,667.0,"""2024-12-31 18:55:48""","""2024-12-31 22:30:59""","""2024-12-31 23:37:00""","""2024-12-31 23:37:00""",1398.416667,241.483333,2152.533333,3653.166667,4453.8,2024.0,"""2024-12-01""","""2024-12-31""","""Wilmington, NC"""


#### Volume Data from Hanouf

In [82]:
#load import data
importvol_df = (
    #read file
    pl.scan_csv('../port data/volumes/portimports.csv', infer_schema_length=0)
    #keep only relevant columns
    .select('CNT_VAL_MO','CNT_WGT_MO','VES_VAL_MO','VES_WGT_MO', 'date', 'matched_PORT_NAME') #only keeping total volumes for now
    #set data types
    .cast({
        'CNT_VAL_MO': pl.Float64, 'CNT_WGT_MO': pl.Float64,
        'VES_VAL_MO': pl.Float64, 'VES_WGT_MO': pl.Float64
    })
    #convert date to month start
    .with_columns(pl.col('date').str.to_date())
    #sum over port and month
    .group_by('date', 'matched_PORT_NAME').sum()
    #housekeeping
    .drop_nulls('matched_PORT_NAME')
    .rename({
        'matched_PORT_NAME': 'port_name', 
        'CNT_VAL_MO': 'val_container_im', 'CNT_WGT_MO': 'wt_container_im',
        'VES_VAL_MO': 'val_total_im', 'VES_WGT_MO': 'wt_total_im'
    })
    .collect()
)

#load export data
exportvol_df = (
    pl.scan_csv('../port data/volumes/portexports.csv', infer_schema_length=0)
    #keep only relevant columns
    .select('CNT_VAL_MO','CNT_WGT_MO','VES_VAL_MO','VES_WGT_MO', 'date', 'matched_PORT_NAME') #only keeping total volumes for now
    #set data types
    .cast({
        'CNT_VAL_MO': pl.Float64, 'CNT_WGT_MO': pl.Float64,
        'VES_VAL_MO': pl.Float64, 'VES_WGT_MO': pl.Float64
    })
    #convert date to date
    .with_columns(pl.col('date').str.to_date())
    #sum over port and month
    .group_by('date', 'matched_PORT_NAME').sum()
    #housekeeping
    .drop_nulls('matched_PORT_NAME')
    .rename({
        'matched_PORT_NAME': 'port_name', 
        'CNT_VAL_MO': 'val_container_ex', 'CNT_WGT_MO': 'wt_container_ex',
        'VES_VAL_MO': 'val_total_ex', 'VES_WGT_MO': 'wt_total_ex'
    })
    .collect()
)

In [83]:
#combine import and export data
volumes_df = (
    importvol_df
    .join(exportvol_df, on=['date', 'port_name'], how='inner', suffix='_export')
    #sum weights and values
    .with_columns(
        (pl.col('val_container_im') + pl.col('val_container_ex'))
        .alias('val_container_total'),
        (pl.col('wt_container_im') + pl.col('wt_container_ex'))
        .alias('wt_container_total'),
        (pl.col('val_total_im') + pl.col('val_total_ex'))
        .alias('val_total'),
        (pl.col('wt_total_im') + pl.col('wt_total_ex'))
        .alias('wt_total')
    )
    #computer bulk (non-container) values and weights for import and export
    .with_columns(
        #totals
        (pl.col('val_total') - pl.col('val_container_total'))
        .alias('val_bulk'),
        (pl.col('wt_total') - pl.col('wt_container_total'))
        .alias('wt_bulk_total'),
        #bulk import and export values and weights
        (pl.col('val_total_ex') - pl.col('val_container_ex'))
        .alias('val_bulk_ex'),
        (pl.col('wt_total_ex') - pl.col('wt_container_ex'))
        .alias('wt_bulk_ex'),
        (pl.col('val_total_im') - pl.col('val_container_im'))
        .alias('val_bulk_im'),
        (pl.col('wt_total_im') - pl.col('wt_container_im'))
        .alias('wt_bulk_im')
    )
    #add port group for NWSPA and San Pedro Ports
    .with_columns(
        pl.when(pl.col('port_name').is_in(['Seattle, WA', 'Tacoma, WA']))
        .then(pl.lit('NW Seaport Alliance'))
        .when(pl.col('port_name').is_in(['Port of Los Angeles, CA',
                                         'Port of Long Beach, CA']))
        .then(pl.lit('San Pedro Ports'))
        .otherwise(pl.col('port_name'))
        .alias('port_group')
    )
    #rename date to month
    .rename({'date': 'month'})
)
#inspect
display(volumes_df.head())
volumes_df.describe()

month,port_name,val_container_im,wt_container_im,val_total_im,wt_total_im,val_container_ex,wt_container_ex,val_total_ex,wt_total_ex,val_container_total,wt_container_total,val_total,wt_total,val_bulk,wt_bulk_total,val_bulk_ex,wt_bulk_ex,val_bulk_im,wt_bulk_im,port_group
date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2022-03-01,"""Jacksonville, FL""",3.0483e9,1.5027e9,9.6621e9,4.1209e9,1.6904e9,3.722987e8,2.7921e9,6.93370158e8,4.7387e9,1.8750e9,1.2454e10,4.8142e9,7.7155e9,2.9392e9,1.1017e9,3.21071458e8,6.6138e9,2.6181e9,"""Jacksonville, FL"""
2020-01-01,"""Corpus Christi, TX""",50000.0,6755.0,3.1208e9,8.2015e9,8.0103156e7,3.79686113e8,2.0740e10,4.6465e10,8.0153156e7,3.79692868e8,2.3861e10,5.4666e10,2.3781e10,5.4287e10,2.0660e10,4.6085e10,3.1208e9,8.2015e9,"""Corpus Christi, TX"""
2018-10-01,"""Milwaukee, WI""",440820.0,177230.0,1.74429622e8,4.72935569e8,140735.0,10456.0,6.0977654e7,1.51031047e8,581555.0,187686.0,2.35407276e8,6.23966616e8,2.34825721e8,6.2377893e8,6.0836919e7,1.51020591e8,1.73988802e8,4.72758339e8,"""Milwaukee, WI"""
2021-09-01,"""New Orleans, LA""",2.3919e9,1.0463e9,8.0942e9,8.4621e9,2.4845e9,1.3775e9,8.0324e9,1.7518e10,4.8764e9,2.4238e9,1.6127e10,2.5980e10,1.1250e10,2.3556e10,5.5479e9,1.6140e10,5.7023e9,7.4158e9,"""New Orleans, LA"""
2021-10-01,"""Port of Oakland, CA""",1.0663e10,2.6104e9,1.1158e10,2.6616e9,7.9951e9,3.2981e9,8.6367e9,3.8935e9,1.8658e10,5.9084e9,1.9795e10,6.5550e9,1.1370e9,6.46607664e8,6.41569957e8,5.95385521e8,4.95422826e8,5.1222143e7,"""Port of Oakland, CA"""


statistic,month,port_name,val_container_im,wt_container_im,val_total_im,wt_total_im,val_container_ex,wt_container_ex,val_total_ex,wt_total_ex,val_container_total,wt_container_total,val_total,wt_total,val_bulk,wt_bulk_total,val_bulk_ex,wt_bulk_ex,val_bulk_im,wt_bulk_im,port_group
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""","""5461""","""5461""",5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,5461.0,"""5461"""
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""","""2021-05-16 02:51:39.652000""",null,6.1433e9,1.2203e9,8.5182e9,3.8517e9,1.7202e9,7.1289e8,4.2676e9,5.4270e9,7.8635e9,1.9332e9,1.2786e10,9.2788e9,4.9223e9,7.3456e9,2.5474e9,4.7142e9,2.3749e9,2.6314e9,null
"""std""",null,null,1.7949e10,3.2557e9,2.0115e10,5.9300e9,3.6772e9,1.5338e9,8.1499e9,1.1666e10,2.1086e10,4.6304e9,2.5209e10,1.5675e10,8.3043e9,1.3843e10,6.2438e9,1.1154e10,3.4673e9,3.6645e9,null
"""min""","""2018-01-01""","""Albany Port District, NY""",0.0,0.0,4140.0,5.0,0.0,0.0,11700.0,624.0,0.0,0.0,23436.0,1709.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Albany Port District, NY"""
"""25%""","""2019-09-01""",null,6.413388e6,1.483942e6,3.30962328e8,5.15599522e8,1.267824e6,329875.0,1.31661624e8,1.84547693e8,2.3116137e7,1.120448e7,7.25399489e8,1.3124e9,4.4174007e8,1.0628e9,6.8818281e7,1.00923005e8,1.87801251e8,4.29459946e8,null
"""50%""","""2021-05-01""",null,1.65980679e8,5.4387745e7,1.5212e9,1.8281e9,6.9431127e7,4.6981721e7,1.1762e9,1.1176e9,3.09513723e8,1.5084177e8,3.6374e9,3.2278e9,1.9778e9,2.5097e9,6.48487957e8,7.27643492e8,9.68760829e8,1.3786e9,null
"""75%""","""2023-02-01""",null,2.4310e9,7.62321801e8,6.4182e9,4.0734e9,1.4240e9,4.06821671e8,4.5838e9,5.2908e9,4.4254e9,1.2121e9,1.1722e10,9.5592e9,5.8243e9,7.0212e9,2.0158e9,3.5139e9,3.0409e9,3.0248e9,null
"""max""","""2024-10-01""","""Wilmington, NC""",1.5166e11,2.5113e10,1.6191e11,3.4814e10,1.9322e10,8.6067e9,7.3539e10,8.9714e10,1.6583e11,3.1019e10,1.7928e11,1.1160e11,7.4756e10,9.7339e10,5.6391e10,8.2669e10,2.2105e10,2.8697e10,"""Wilmington, NC"""


In [85]:
#construct columns dictionary for volumes_df
volumes_col_descriptions = {
    'val_container_im': 'Value of Containerized Imports',
    'val_container_ex': 'Value of Containerized Exports',
    'val_container_total': 'Value of Total Containerized Cargo',
    'val_bulk': 'Value of Bulk Cargo',
    'val_bulk_ex': 'Value of Bulk Exports',
    'val_bulk_im': 'Value of Bulk Imports',
    'val_total': 'Value of Total Cargo',
    'val_total_ex': 'Value of Total Exports',
    'val_total_im': 'Value of Total Imports',
    'wt_container_im': 'Weight of Containerized Imports',
    'wt_container_ex': 'Weight of Containerized Exports',
    'wt_container_total': 'Weight of Total Containerized Cargo',
    'wt_bulk': 'Weight of Bulk Cargo',
    'wt_bulk_ex': 'Weight of Bulk Exports',
    'wt_bulk_im': 'Weight of Bulk Imports',
    'wt_total': 'Weight of Total Cargo',
    'wt_total_ex': 'Weight of Total Exports',
    'wt_total_im': 'Weight of Total Imports',
    'date': 'Date',
    'port_name': 'Port Name'
}

In [86]:
fig = px.line(exportvol_df.sort('port_name','date'), x='date', y='val_total_ex', 
        title='Seattle Export Value In Context', color='port_name')
fig.update_traces(line_color='lightgray')
fig.update_traces(patch={'line': {'color': 'blue'}}, 
                          selector=dict(name='Seattle, WA'))
fig.show()

In [89]:
fig = px.line(volumes_df.sort('port_name','month'), x='month', y='val_total_im', 
        title='Seattle Import Value In Context', color='port_name')
fig.update_traces(line_color='lightgray')
fig.update_traces(patch={'line': {'color': 'blue'}}, 
                          selector=dict(name='Seattle, WA'))
fig.show()

In [90]:
fig = px.line(volumes_df.sort('port_name','month'), x='month', y='val_container_im', 
        title='Containerize Value (imports)', color='port_name')
fig.update_traces(line_color='lightgray')
fig.update_traces(patch={'line': {'color': 'blue'}}, 
                          selector=dict(name='Port of Long Beach, CA'))
fig.show()

In [ ]:
fig = px.line(volumes_df.sort('port_name','date'), x='date', y='wt_total_im', 
        title='Total Weight (imports)', color='port_name')
fig.update_traces(line_color='lightgray')
fig.update_traces(patch={'line': {'color': 'blue'}}, 
                          selector=dict(name='Port of Los Angeles, CA'))
fig.show()

## Visualization Functions

For the dashboard and PNREC presentation, we define a handful of visualization types (e.g. scatter map, line plot, bar chart, etc) and code to allow both default visualizations as well as custom user-generated visualizations. 

In [134]:
#define zoom level function for plotly express scatter_mapbox
def mapbox_zoom_finder(lons, lats, lon_pad=0, lat_pad=0):
    """
    Calculates the optimal zoom level for a Plotly Mapbox plot.
    Args:
        lons (list): List of longitudes.
        lats (list): List of latitudes.
        lon_pad (float, optional): Padding to add to the longitude range. Defaults to 0.
        lat_pad (float, optional): Padding to add to the latitude range. Defaults to 0.
    Returns:
        zoom (int): the calculated zoom level
    """
    # Check if the lengths of lons and lats are equal and not empty
    if len(lons) != len(lats) or len(lons) == 0:
        return 10
    # Calculate the maximum and minimum longitude and latitude
    max_lon, min_lon = max(lons), min(lons)
    max_lat, min_lat = max(lats), min(lats)
    # Calculate the longitude and latitude ranges
    lon_range = max_lon - min_lon
    lat_range = max_lat - min_lat
    # Calculate the zoom level based on the ranges
    zoom = 7 - np.log2(max(lon_range + lon_pad, lat_range + lat_pad))
    return zoom

def plot_mapbox(df, cat_group,  lat_col, lon_col, 
                size_col, size_col_alias, color_col, color_col_alias, 
                title, filter_col=None, filter=None, time_col='date', 
                time_range=[earliest_date, latest_date], 
                zoom=None, center=None, width=800, height=600, 
                size_max=30, range_color=None, hover_name=None, hover_data=None, 
                mapbox_style='carto-positron', labels=None, 
                color_continuous_scale=None, color_outlier_z=None):
    """
    Plots a Mapbox scatter plot using Plotly.
    """
    #init df
    df = df
    #filter if specified
    if filter_col:
        df = df.filter(pl.col(filter_col).is_in(filter))
    #generate df
    df = (
        df
        #filter by time range
        .filter(pl.col(time_col).is_between(time_range[0], time_range[1]))
        .group_by(cat_group)
        .agg(
            #get the lat and lon columns
            pl.col(lat_col).first().alias(lat_col),
            pl.col(lon_col).first().alias(lon_col),
            #get hover name
            #may be different or same as cat_group
            #get stats
            pl.col(size_col).mean().alias(size_col_alias),
            pl.col(color_col).mean().alias(color_col_alias),
        )
    )
    #Set default color scale if not provided
    if not color_continuous_scale:
        color_continuous_scale = px.colors.sequential.Viridis

    # Set the zoom level automatically if not provided
    if not zoom:
        zoom = mapbox_zoom_finder(df[lon_col], df[lat_col])

    #Set the center of the map if not provided
    if not center:
        center = {
            'lat': ((df[lat_col].max() - df[lat_col].min()) / 2),
            'lon': ((df[lon_col].max() - df[lon_col].min()) / 2)
        }

    #drop outliers if specified
    if color_outlier_z:
        #get color_col upper and lower limits based on z score
        color_col_mean, color_col_std = df[color_col_alias].mean(), df[color_col_alias].std()
        color_col_upper = color_col_mean + (color_col_std * color_outlier_z)
        color_col_lower = color_col_mean - (color_col_std * color_outlier_z)
        #set range color
        range_color = [color_col_lower, color_col_upper]

    # Create a scatter mapbox figure
    fig = px.scatter_mapbox(
        #data
        df, lat=lat_col, lon=lon_col,
        #categories
        size=size_col_alias, color=color_col_alias,
        #hover info
        hover_name=hover_name, hover_data=hover_data,
        #display settings
        range_color=range_color, size_max=size_max,
        color_continuous_scale=color_continuous_scale, mapbox_style=mapbox_style,
        width=width, height=height,
        #title and labals
        title=title, labels=labels
    )
    # Set the zoom level
    fig.update_layout(mapbox_zoom=zoom)

    #NOTE Add annotation if specified

    return fig

def plot_line(df, cat_group, time_col, y_col, y_col_alias, title,
              filter_col=None, filter=None, cat_limit=None, cat_limit_col=None,
              stat='mean', time_conversion=None,
              time_range=[earliest_date, latest_date], 
              volume_df=None, volume_col=None,
              width=800, height=600, hover_name=None, 
              hover_data=None, labels=None, color_continuous_scale=None, 
              highlight=None, highlight_color=None):
    """
    Plots a line chart using Plotly Express.
    """
    #initialize df
    df = (
        df
        #filter by time range
        .filter(pl.col(time_col).is_between(time_range[0], time_range[1]))
    )
    #filter if specified
    if filter_col:
        df = df.filter(pl.col(filter_col).is_in(filter))
    #generate df
    df = (
        df
        .group_by(cat_group, time_col)
        .agg(
            #compute y col mean
            pl.col(y_col).mean().alias(y_col+'_mean'),
            #compute y col sum
            pl.col(y_col).sum().alias(y_col+'_sum'),
        )
        .sort(time_col)
    )
    #rename y col to match stat choice
    if stat == 'sum':
        df = df.rename({y_col+'_sum': y_col_alias})
    elif stat == 'mean':
        df = df.rename({y_col+'_mean': y_col_alias})
    else:
        raise ValueError(f"Invalid stat: {stat}. Must be 'sum' or 'mean'.")
    #merge in volume data if specified
    if volume_df is not None:
        #merge in volume data
        df = (
            df
            .join(volume_df.group_by(cat_group, time_col)
                  .agg(pl.col(volume_col).sum().alias(volume_col)), 
                  on=[time_col, cat_group], how='inner')
            #compute y column 
            .with_columns(
                (pl.col(volume_col) / pl.col(y_col_alias))
                .alias(y_col_alias)
            )
            .sort(time_col)
        )
    #limit categories if specified
    if cat_limit:
        #get the top n categories
        top_cats = (
            df.group_by(cat_group)
            .agg(pl.col(cat_limit_col).sum())
            .sort(pl.col(cat_limit_col), descending=True)
            .limit(cat_limit)
            .to_series()
        )
        #filter to only top n categories or highlight category
        df = (
            df.filter(pl.col(cat_group)
                       .is_in(top_cats.append(pl.Series([highlight]))))
            .sort('month')
        )
    #convert time if specified
    if time_conversion:
        #convert time column to datetime
        df = df.with_columns(
            pl.col(y_col_alias)*time_conversion
        )

    # Set default color scale if not provided
    if not color_continuous_scale:
        color_continuous_scale = px.colors.sequential.Viridis

    # Create a line figure
    fig = px.line(
        df,
        x=time_col,
        y=y_col_alias,
        color=cat_group,
        title=title,
        labels=labels,
        hover_name=hover_name,
        hover_data=hover_data,
    )
    # Set the width and height of the figure
    fig.update_layout(width=width, height=height)

    #highlight given lines if specified
    if highlight:
        fig.update_traces(line_color='lightgray')
        fig.update_traces(patch={'line': {'color': highlight_color}}, 
                          selector=dict(name=highlight))

    return fig

#the above takes the mean of y_col; this takes the count (e.g., number of calls)
def plot_line_count(df, cat_group, time_col, y_col, y_col_alias, title,
              filter_col=None, filter=None, cat_limit=None, cat_limit_col=None, 
              time_range=[earliest_date, latest_date], 
              width=800, height=600, hover_name=None, 
              hover_data=None, labels=None, color_continuous_scale=None, 
              highlight=None, highlight_color=None):
    """
    Plots a line chart using Plotly Express.
    """
    #initialize df
    df = (
        df
        #filter by time range
        .filter(pl.col(time_col).is_between(time_range[0], time_range[1]))
    )
    #filter if specified
    if filter_col:
        df = df.filter(pl.col(filter_col).is_in(filter))
    #limit categories if specified
    if cat_limit:
        #get the top n categories
        top_cats = (
            df.group_by(cat_group)
            .agg(pl.col(cat_limit_col).count())
            .sort(pl.col(cat_limit_col), descending=True)
            .limit(cat_limit)
            .to_series()
        )
        #filter to only top n categories or highlight category
        df = df.filter(pl.col(cat_group)
                       .is_in(top_cats.append(pl.Series([highlight]))))
    #generate df
    df = (
        df
        .group_by(cat_group, time_col)
        .agg(
            #compute y col mean
            pl.col(y_col).count().alias(y_col_alias)
        )
        .sort(time_col)
    )

    # Set default color scale if not provided
    if not color_continuous_scale:
        color_continuous_scale = px.colors.sequential.Viridis

    # Create a line figure
    fig = px.line(
        df,
        x=time_col,
        y=y_col_alias,
        color=cat_group,
        title=title,
        labels=labels,
        hover_name=hover_name,
        hover_data=hover_data,
    )
    # Set the width and height of the figure
    fig.update_layout(width=width, height=height)

    #highlight given lines if specified
    if highlight:
        fig.update_traces(line_color='lightgray')
        fig.update_traces(patch={'line': {'color': highlight_color}}, 
                          selector=dict(name=highlight))

    return fig


def bar_ranking(df, cat_group, stat_col, stat_alias, title, limit=20, 
                filter_col=None, filter=None, 
                time_col='month', time_range=[earliest_date, latest_date],
                labels=None, width=800, height=600, 
                highlight=None):
    '''
    '''
    #initialize df
    df = (
        df
        #filter by time range
        .filter(pl.col(time_col).is_between(time_range[0], time_range[1]))
    )
    #filter if specified
    if filter_col:
        df = df.filter(pl.col(filter_col).is_in(filter))
    #get top n categories
    top_cats = (
        df
        .group_by(cat_group)
        .agg(pl.col(stat_col).mean().alias(stat_alias))
        .sort(pl.col(stat_alias), descending=True)
        .limit(limit)
        .to_series()
    )
    #create df
    df = (
        df
        .filter(pl.col(cat_group).is_in(top_cats.append(pl.Series([highlight]))))
        .group_by(cat_group)
        .agg(pl.col(stat_col).mean().alias(stat_alias))
        .sort(pl.col(stat_alias))
    )
    # Create a bar figure
    fig = px.bar(
        df,
        x=stat_alias, y=cat_group,
        title=title, labels=labels,
        width=width, height=height,
    )

    #set highlight if specified
    if highlight:
        fig["data"][0]["marker"]["color"] = (
            [fig["data"][0]["marker"]['color'] if c == highlight 
             else "lightgrey" for c in fig["data"][0]["y"]]
        )
    
    return fig


## PNREC Visualizations

### National Context

To put the Northwest Seaport Alliance in context, we present various KPIs and associated trends


In [93]:
plot_line_count(
        df=calls_df, cat_group='port_group', time_col='month',
        y_col='hrs_at_berth', y_col_alias='Number of Calls per Month',
        title='Total Calls at Principal Ports', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='hrs_at_berth',
        highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [97]:
plot_line(
        df=calls_df, cat_group='port_group', time_col='month',
        y_col='vessel_size', y_col_alias='Mean Vessel Length (m)',
        title='Vessel Size at Top Ports', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='Mean Vessel Length (m)',
        highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [96]:
plot_line(
        df=calls_df.filter(pl.col('port_group').is_in(['NW Seaport Alliance', 'Virginia, VA, Port of'])),
        cat_group='port_group', time_col='month',
        y_col='vessel_size', y_col_alias='Mean Vessel Length (m)',
        title='Vessel Size - Diverging Trends', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='Mean Vessel Length (m)',
    )

#### Efficiency and Performance

In [141]:
plot_line(
        df=calls_df, cat_group='port_group', time_col='month',
        y_col='hrs_at_berth', y_col_alias='Average Hours at Berth',
        title='Hours at Berth per call at Principal Ports', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='Average Hours at Berth',
        highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [142]:
plot_line(
        df=calls_df, cat_group='port_group', time_col='month',
        y_col='hrs_at_berth', y_col_alias='Efficiency ($/hr at berth)',
        stat='sum',
        volume_df=volumes_df, volume_col='val_total_im',
        title='Dollars of Trade Value per Vessel-Hour at Berth at Principal Ports', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=5, cat_limit_col='val_total_im',
        highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [140]:
fig = px.line(volumes_df
              .group_by('port_group','month').agg(pl.col('val_total_im').sum())
              .sort('port_group','month'), 
              x='month', y='val_total_im', 
              title='NWSPA Import Value In Context', color='port_group')
fig.update_traces(line_color='lightgray')
fig.update_traces(patch={'line': {'color': 'blue'}}, 
                          selector=dict(name='NW Seaport Alliance'))
fig.show()

In [ ]:
plot_line(
        df=calls_df, cat_group='port_group', time_col='month',
        y_col='hrs_at_berth', y_col_alias='Efficiency (ms/kg)',
        stat='sum', time_conversion=3600000,
        volume_df=volumes_df, volume_col='wt_total',
        title='Time at Berth per Kg Traded at Principal Ports', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='wt_total',
        highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [ ]:
plot_line(
        df=calls_df, cat_group='port_group', time_col='month',
        filter_col='port_group', filter=['NW Seaport Alliance', 'San Pedro Ports'],
        y_col='hrs_at_berth', y_col_alias='Average Hours at Berth',
        title='Hours at Berth per call - San Pedro vs NWSPA', width=None, height=None,
        labels={'port_group':'Port', 'month':''},
        cat_limit=10, cat_limit_col='hrs_at_berth',
        #highlight='NW Seaport Alliance', highlight_color='blue'
    )

In [ ]:
#group and define efficiency metrict
df = (
    calls_df
    .with_columns(
        (pl.col('vessel_size')/pl.col('hrs_at_berth')).alias('Efficiency Score')
    )
)

bar_ranking(
    df,
    cat_group='port_group',
    stat_col='Efficiency Score',
    stat_alias='Vessel Size (m) per Hour at Berth',
    title='Port Efficiency: Vessel Size per Hour at Berth',
    limit=30,
    labels={'port_group':'Port'},
    highlight='NW Seaport Alliance',
    width=800,
    height=600,
)

better: total weight per hour at berth

In [ ]:
#get top 20 ports by calls
top_groups_calls = (
    calls_df
    .group_by('port_group')
    .agg(pl.col('call_id').count().alias('total_calls'))
    .sort(pl.col('total_calls'), descending=True)
    .limit(20)
)

#plot
bar_ranking(df=calls_df, cat_group='port_group', stat_col='hrs_in_port_waters',
            filter_col='port_group', filter=top_groups_calls['port_group'],
            stat_alias='Time in Port Waters (hrs)', 
            title='Average Time in Port Waters (Top 20 ports by total calls)',
            labels={'port_group':''}, highlight='NW Seaport Alliance',
            limit=50, width=800, height=500)

### Dock-level Visualizations

In [ ]:
plot_mapbox(df=calls_df, cat_group='dock_name', filter_col='port_name', 
            filter=['Tacoma, WA'],
            lat_col='dock_lat', lon_col='dock_lon', size_col='hrs_at_berth', 
            size_col_alias='Mean Hours at Berth',
            color_col='hrs_at_anchor', color_col_alias='Mean Hours at Anchor',
            size_max=20, color_outlier_z=2,
            title='Hours at Berth vs. Hours at Anchor - Tacoma', 
            hover_name='dock_name')

Schema({'dock_name': String, 'dock_lat': Float64, 'dock_lon': Float64, 'Mean Hours at Berth': Float64, 'Mean Hours at Anchor': Float64})


/var/folders/dc/sgfd2dls28n4y4v4_jw03ly40000gp/T/ipykernel_5434/2657556165.py:103: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [ ]:
plot_mapbox(df=calls_df, cat_group='dock_name', filter_col='port_name', 
            filter=['Seattle, WA'],
            lat_col='dock_lat', lon_col='dock_lon', size_col='hrs_at_berth', size_col_alias='Mean Hours at Berth',
            color_col='hrs_at_anchor', color_col_alias='Mean Hours at Anchor',
            size_max=20, color_outlier_z=2,
            title='Hours at Berth vs. Hours at Anchor - Seattle', hover_name='dock_name')

Note current issue with Tacoma Docks not showing up - will fix and re-run. 

In [ ]:
plot_mapbox(df=calls_df, cat_group='dock_name', filter_col='port_group', 
            filter=['San Pedro Ports'],
            lat_col='dock_lat', lon_col='dock_lon', size_col='hrs_at_berth', size_col_alias='Mean Hours at Berth',
            color_col='hrs_at_anchor', color_col_alias='Mean Hours at Anchor',
            size_max=20, color_outlier_z=2, zoom=11.5,
            title='Hours at Berth vs. Hours at Anchor - San Pedro Ports', hover_name='dock_name')

In [ ]:
#group and define efficiency metrict
df = (
    calls_df.filter(pl.col('port_group')=='NW Seaport Alliance')
    .with_columns(
        (pl.col('vessel_size')/pl.col('hrs_at_berth')).alias('Efficiency Score')
    )
)

bar_ranking(df,
            cat_group='dock_name', stat_col='Efficiency Score',
            stat_alias='Vessel Size (m) per Hour at Berth', 
            title='Top Performing Docks - NW Seaport Alliance',
            labels={'dock_name':''},
            limit=20, width=800, height=500)